The first section only have exploratory trials...

In [12]:
## This function will use CKAN api to get packages list plus the url for the resources under each entity 
## The results are stored in a pandas DF

import pprint
import requests
import pandas as pd

def getPackageList(url_api):
    r = requests.get(url_api)
    j = r.json()
    return(pd.DataFrame({"package_id": j['result']}))

def getResourceUrl(package_id):
    url = "https://opendata.porto.hackacity.eu" + "/api/3/action/package_show?id=" + package_id;  
    r = requests.get(url)
    j = r.json()
    return(j['result']['resources'][0]['url'])

In [ ]:
## Usage

url_api = "https://opendata.porto.hackacity.eu/api/3/action/package_list"
df = getPackageList(url_api)

df["resource_url"] = list(map(getResourceUrl, df["package_id"]))

#print(df)

In [4]:
## Get packageIds as a list

def getPackageIds(url):
    r = requests.get(url)
    j = r.json()
    return(j['result'])

In [ ]:
## Usage

url = "https://opendata.porto.hackacity.eu/api/3/action/package_list";
packageIdsList = getPackageIds(url)
#print(packageIdsList)

In [279]:
## For a list of packagesIds extract the available metadata and store the results in a pandas DF

from pandas.io.json import json_normalize

def normalizeJSON(url, package_id):
    url = url + package_id;  
    r = requests.get(url)
    j = r.json()
    return(json_normalize(j['result']))

def appendRowToDF(df1, df2):
    df = pd.DataFrame()
    if df1.empty: 
        df = df1.copy()
    else:
        df = pd.concat([df1,df2])
    return(df)

In [ ]:
## Usage

packagesMetadata = pd.DataFrame()
url = "https://opendata.porto.hackacity.eu/api/3/action/package_show?id="
for packageId in packageIdsList:
    row = normalizeJSON(url, packageId)
    packagesMetadata = appendRowToDF(packagesMetadata, row)  

In [45]:
## Request live data and store the result in a panda DF

import requests
import pprint
import pandas
from pandas.io.json import json_normalize

def getLiveData(api_url, package_id = None, device_id = None):
    if (package_id is None) and (device_id is not None):
        url = api_url + "/v2/entities?id=" + device_id
    elif (package_id is not None) and (device_id is None):
        url = api_url + "/v2/entities?type=" + package_id
    else:
        raise Exception("package_id or device_id required or are simultaneous defined!")

    print(url)
    
    r = requests.get(url)
    j = r.json()
    #pprint.pprint(j)
    return(json_normalize(j))

In [256]:
## Example with error
#api_url = "https://broker.fiware.urbanplatform.portodigital.pt"
#df = getLiveData(api_url)

In [46]:
## Example without device id
api_url = "https://broker.fiware.urbanplatform.portodigital.pt"
package_id = "AirQualityObserved"
df = getLiveData(api_url, package_id)

https://broker.fiware.urbanplatform.portodigital.pt/v2/entities?type=AirQualityObserved


In [51]:
## Example with device id
api_url = "https://broker.fiware.urbanplatform.portodigital.pt"
device_id = "urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5adf39366f555a4514e7ea54"
df = getLiveData(api_url, device_id = device_id)

https://broker.fiware.urbanplatform.portodigital.pt/v2/entities?id=urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5adf39366f555a4514e7ea54


In [14]:
## Example 
## require getHistoricalData function declared below!!!

#resource_list = ['urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5adf39366f555a4514e7ea54', 
#                 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b97a1bde521e3053085c08c', 
#                 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b632f2706599b05e998bed8',
#                 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b72dbfa06599b05e9a74f27',
#                 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b97a1a6e521e3053085c067']

#api_url = "http://history-data.urbanplatform.portodigital.pt"

#df = pd.DataFrame()
#for resource in resource_list:
#    print(resource)
#    df1 = getHistoricalData(api_url, device_id = resource, no=500)
#    #print(df1.columns)
#    if df.empty:
#        df = df1.copy()
#    else:
#        df = pd.concat([df,df1])
#    #print(df.columns)   


From this point forward are the sequential steps to get the IOT data...


In [16]:
## List IOT entities
import requests
import json

def getListIOTEntities(api_url):
    url = api_url + "/v2/types"
    r = requests.get(url)
    j = r.json()
    entity_list = []
    for item in j:
        #print(item['type'])
        entity_list.append(item['type'])
    return(entity_list)

In [17]:
api_url = "https://broker.fiware.urbanplatform.portodigital.pt"
entity_list = getListIOTEntities(api_url)
print(entity_list)

['AirQualityObserved', 'ArrivalEstimation', 'Device', 'Event', 'GtfsTransitFeedFile', 'NoiseLevelObserved', 'OffStreetParking', 'PointOfInterest', 'TrafficFlowObserved', 'Vehicle', 'WeatherForecast', 'WeatherObserved', 'gtfsAgency', 'gtfsCalendarDateRule', 'gtfsCalendarRule', 'gtfsRoute', 'gtfsService', 'gtfsStop']


In [18]:
## Reduce entity_list to the ones of interest (MANUAL STEP)
entity_list = ['AirQualityObserved']

In [19]:
## Get list of all available resources/devices for an entity

import requests
import pprint

def getResourceList(api_url, entity_name):
    # First let's the ID's from the FiWare
    url = api_url + "/v2/entities?type=" + entity_name
    #print(url)
    r = requests.get(url, verify=False)
    j = r.json()
    #print(j)
    resources_list = []
    for resource in j: 
      resources_list.append(resource['id'])
    return(resources_list)

In [20]:
api_url = "https://broker.fiware.urbanplatform.portodigital.pt"

resource_list = []
for entity in entity_list:
    #print(entity)
    resource_list.append(getResourceList(api_url, entity))    

/home/cha1av/dev/hackacity/env/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
#print(resource_list)

In [21]:
#Flat resource_list
resource_list_flat = [item for sublist in resource_list for item in sublist]

In [23]:
#Remove stuff that is giving errors
#resource_list_flat.remove('Trindade_1_Rotacao')

In [24]:
print(resource_list_flat)

['urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5adf39366f555a4514e7ea54', 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b97a1bde521e3053085c08c', 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b632f2706599b05e998bed8', 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b72dbfa06599b05e9a74f27', 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b97a1a6e521e3053085c067']


In [58]:
## Request historical data with resource_id and store the results as a panda DF for a single device

import requests
import pprint
import pandas
from pandas.io.json import json_normalize

def convertJSONHistoricalToDF(j):
    df = pd.DataFrame()
    for i in range(0,len(j['data']['attributes'])):
        df[j['data']['attributes'][i]['attrName']] = j['data']['attributes'][i]['values']
    ## expand location attributes
    #for key in df['location'][0].keys():
    #    df[key] = str(df['location'][0][key])
    df['long'] = df['location'][0]['coordinates'][0]
    df['lat'] = df['location'][0]['coordinates'][1]
    df['device_id'] = j['data']['entityId']
    #print(df)
    return(df)
    
def getHistoricalData(api_url, device_id = None, no = 20):
    if (device_id is not None):
        url = api_url + "/v2/entities/" + device_id + "?limit=" + str(no)
    else:
        raise Exception("device_id is required")

    #print(url)
    r = requests.get(url)
    j = r.json()
    #json_normalize(j['data'], record_path=['attributes']).transpose()
    return(convertJSONHistoricalToDF(j))

In [59]:
api_url = "http://history-data.urbanplatform.portodigital.pt"

df = pd.DataFrame()
for resource in resource_list_flat:
    print(resource)
    df1 = getHistoricalData(api_url, device_id = resource, no=500)
    #print(df1.columns)
    if df.empty:
        df = df1.copy()
    else:
        df = pd.concat([df,df1], sort=False)
    #print(df.columns)  

urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5adf39366f555a4514e7ea54
urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b97a1bde521e3053085c08c
urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b632f2706599b05e998bed8
urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b72dbfa06599b05e9a74f27
urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b97a1a6e521e3053085c067


In [31]:
#pprint.pprint(df)

In [61]:
# DF operations

import  time
import datetime

# Convert timestamp to unixtime
def convert_df_to_unix(s):
    time_mask = "%Y-%m-%dT%H:%M:%S.%f"
    return(time.mktime(datetime.datetime.strptime(s, time_mask).timetuple()))
 


In [62]:
df['time'] = list(map(convert_df_to_unix, df['dateObserved']))

In [65]:
## Save columns to csv in a custom order
mandatory_column_list = ["time", "lat", "long"]
data_column_list = ["CO", "NO2", "O3", "Ox", "PM1", "PM10", "PM25"]
for data_column in data_column_list:
    filepath = "./output/" + "IOT_" + "AirQuality_" + data_column + ".csv"
    print(filepath)
    df.to_csv(filepath, columns = mandatory_column_list + [data_column], 
              index = False, na_rep = "", header = True)

./output/IOT_AirQuality_CO.csv
./output/IOT_AirQuality_NO2.csv
./output/IOT_AirQuality_O3.csv
./output/IOT_AirQuality_Ox.csv
./output/IOT_AirQuality_PM1.csv
./output/IOT_AirQuality_PM10.csv
./output/IOT_AirQuality_PM25.csv
